# **Redes Quânticas - Exemplo 1: Conexões**

Neste notebook nós praticamos a utilização de conexões entre os nós, para que possamos modelar mais facilmente situações mais complexas (por exemplo, fornecer emaranhamento através de uma fonte quântica)

In [1]:
import netsquid as ns
import pandas
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.nodes import Node, Connection, Network
from netsquid.protocols.protocol import Signals
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.components.qchannel import QuantumChannel
from netsquid.components.cchannel import ClassicalChannel
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components.qprogram import QuantumProgram
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel
import netsquid.components.instructions as instr
import pydynaa

## **Exemplo 1: Conexões**

Já praticamos como conectar dois nós através de um canal clássico e/ou quântico. Em vez de conectar canais diretamente aos nós, agora praticaremos como usar conexões.

Começamos com uma conexão quântica e usaremos o canal quântico para enviar um qubit de um nó para outro. 


Primeiro definimos as instruções físicas e inicializamos dois processadores quânticos com uma posição de memória cada.


In [2]:
phys_instructions1 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_H, duration=1),
    PhysicalInstruction(instr.INSTR_X, duration=1),]

qproc1a = QuantumProcessor(name="ExampleQProc1a", num_positions =1, phys_instructions = phys_instructions1)
qproc1b = QuantumProcessor(name="ExampleQProc1b", num_positions =1, phys_instructions = phys_instructions1)

<br><br>Como antes, cada processador será atribuído a um nó. Os dois nós estarão fisicamente separados um do outro e nosso objetivo é enviar um qubit de um nó para o outro. Para fazer isso, inicializaremos os dois nós com portas para comunicação de entrada (qin) e saída (qout) e conectaremos as portas de memória às portas do nó.

In [3]:
node1a = Node(name="ExampleNode1a", qmemory=qproc1a, port_names=['qin_1a','qout_1a'])
node1b = Node(name="ExampleNode1b", qmemory=qproc1b, port_names=['qin_1b','qout_1b'])

node1a.qmemory.ports['qout0'].forward_output(node1a.ports['qout_1a'])
node1b.ports['qin_1b'].forward_input(node1b.qmemory.ports['qin0'])

<br><br>No notebook anterior, inicializamos em seguida um canal quântico que usamos para conectar os dois nós entre si. Aqui, definimos agora uma conexão quântica que contém um canal quântico.

<span style = "color:orange"> class netsquid.nodes.connections.Connection(name) </span>

In [4]:
class QuantumConnection(Connection):
    """ Uma conexão que neste exemplo contém apenas um canal quântico. 
    No entanto, podemos incluir uma fonte quântica desde a conexão 
    e usá-la para distribuir qubits emaranhados aos nós conectados."""
    
    def __init__(self, name = "QuantumConnection"):
        super().__init__(name=name)
        
        # Inicializando um canal quântico
        qchannell = QuantumChannel(name="QChannel_1a21b")
        
        # Conectando o canal para entrada e saída das portas da conexão
        self.add_subcomponent(qchannell, 
                             forward_input=[("A","send")],
                             forward_output=[("B","recv")])

<br><br>Agora, conectamos as portas dos nós às portas da conexão.

In [5]:
qconn = QuantumConnection()
node1a.ports['qout_1a'].connect(qconn.ports['A'])
node1b.ports['qin_1b'].connect(qconn.ports['B'])

<br><br>Em seguida, nós definimos um programa quântico para o node1a que
- incializa um qubit na posição 0 no node1a
- mudam o estado do qubit para $|+\rangle = \frac{1}{\sqrt 2}(|0\rangle + |1\rangle)$


Um protocolo para node1a que
- executa o programa
- remove o qubit da memória
- envia o qubit para a porta de saída da memória 

e um protocolo para o node1b que apenas aguarde pelo qubit recebido

In [6]:
class node1aProgram(QuantumProgram):
    default_num_qubits=1 # Numero de qubits
    
    def program(self):
        q0, = self.get_qubit_indices(1) # Obtém os indices para o qubit que estamos trabalhando
        self.apply(instr.INSTR_INIT, [q0]) # Inicializa o qubit
        
        # Define o estado do qubit para h0
        self.apply(instr.INSTR_H, [q0]) # Aplica a porta H em q0
        
        yield self.run() # Executa as instruções agendada acima
        
class node1aProtocol(NodeProtocol):
    def run(self):
        node1aprog = node1aProgram()
            
        yield self.node.qmemory.execute_program(node1aprog)

        qubit = self.node.qmemory.pop(positions=[0])
        self.node.qmemory.ports["qout0"].tx_output(qubit)

class node1bProtocol(NodeProtocol):
    def run(self):
        port_qin = self.node.ports["qin_1b"]
        
        # Aguardando o qubit chegar aqui
        expr = yield(self.await_port_input(port_qin))

<br><br>Tudo o que resta fazer é executar os dois protocolos

In [7]:
node1aprot = node1aProtocol(node1a)
node1bprot = node1bProtocol(node1b)

node1aprot.start() # Inicia o protocolo no node1a
node1bprot.start() # Inicia o protocolo no node1b

ns.sim_run() # Inicia a simulação
ns.sim_time()

4.0

Espiando as memória, vemos que o qubit foi movido conforme planejado

In [8]:
node1b.qmemory.peek(positions=[0])

[Qubit('QS#0-0')]

In [9]:
qubit1b, = node1b.qmemory.peek(positions=[0])
print("state of qubit1b as ket", qubit1b.qstate.qrepr)

state of qubit1b as ket KetRepr(num_qubits=1,
ket=
[[0.70710678+0.j]
 [0.70710678+0.j]])
